In [55]:
import os
os.environ['LANGCHAIN_API_KEY']="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"
import json
from langsmith import Client
from uuid import UUID
import pandas as pd
client = Client()
from datetime import datetime, timedelta

In [80]:
start_time = datetime.utcnow() - timedelta(days=7)
project_name = "sphinx-doc__sphinx-9591"

runs = list(
    client.list_runs(
        project_name=project_name,
        run_type="llm",
        # start_time=start_time,
    )
)

In [86]:
df = pd.DataFrame(
    [
        {
            "trace_id": run.trace_id,
            "name": run.name,
            "model": run.extra["invocation_params"][
                "model_id"
            ],  # The parameters used when invoking the model are nested in the extra info
            **run.inputs,
            **(run.outputs or {}),
            "start_time": run.start_time,
            "error": run.error,
            "latency": (run.end_time - run.start_time).total_seconds()
            if run.end_time
            else None,  # Pending runs have no end time
            "prompt_tokens": run.prompt_tokens,
            "completion_tokens": run.completion_tokens,
            "total_tokens": run.total_tokens,
            "metadata": run.metadata,
        }
        for run in runs
    ],
    index=[run.id for run in runs],
)

In [85]:
runs[0].extra['invocation_params']["model_id"]

'anthropic.claude-3-5-sonnet-20240620-v1:0'

In [32]:
df.to_excel("runs.xlsx")

In [33]:
df.to_csv("runs.csv")

In [34]:
run = runs[0]

In [35]:
run.status

'success'

In [43]:
for _,_df in df.groupby(by="trace_id"):
    print(_df.iloc[0]["name"])
    print(len(_df))
    print("-"*100)
    # break


BedrockChat
1
----------------------------------------------------------------------------------------------------
BedrockChat
1
----------------------------------------------------------------------------------------------------
BedrockChat
1
----------------------------------------------------------------------------------------------------
BedrockChat
3
----------------------------------------------------------------------------------------------------
BedrockChat
38
----------------------------------------------------------------------------------------------------
BedrockChat
1
----------------------------------------------------------------------------------------------------
BedrockChat
17
----------------------------------------------------------------------------------------------------
BedrockChat
24
----------------------------------------------------------------------------------------------------
BedrockChat
1
---------------------------------------------------------------

In [50]:
rundict_dir = "./logs/run_dicts"

os.makedirs(rundict_dir, exist_ok=True)
os.makedirs(f"{rundict_dir}/{project_name}", exist_ok=True)
runs_dict = []
for run in runs:
    run_data = run.__dict__.copy()
    # Convert UUID objects to strings
    for key, value in run_data.items():
        if isinstance(value, UUID):
            run_data[key] = str(value)
    runs_dict.append(run_data)

with open(f"{rundict_dir}/{project_name}/runs_dict.json", "w") as f:
    json.dump(runs_dict, f, indent=4, default=str)


In [54]:
from easydict import EasyDict
with open(f"{rundict_dir}/{project_name}/runs_dict.json", "r") as f:
    runs_dict = json.load(f)
runs_dict = [EasyDict(run) for run in runs_dict]

In [62]:
df = pd.DataFrame(
    [
        {
            "trace_id": run.trace_id,
            "name": run.name,
            # "model": run.extra["invocation_params"][
            #     "model"
            # ],  # The parameters used when invoking the model are nested in the extra info
            **run.inputs,
            **(run.outputs or {}),
            "start_time": run.start_time,
            "error": run.error,
            # "latency": (run.end_time - run.start_time).total_seconds()
            # if run.end_time
            # else None,  # Pending runs have no end time
            "prompt_tokens": run.prompt_tokens,
            "completion_tokens": run.completion_tokens,
            "total_tokens": run.total_tokens,
            # "metadata": run.metadata,
        }
        for run in runs_dict
    ],
    index=[run.id for run in runs_dict],
)

In [64]:
for _,_df in df.groupby(by="trace_id"):
    print(len(_df))


1
1
1
3
38
1
17
24
1
1
3
39
37
1
1
3
37


In [66]:
_df_sorted = _df.sort_values(by="start_time")


In [71]:
for i, row in _df_sorted.iterrows():
    print(row["messages"])
    print(row["llm_output"])
    print("-"*100)
    break


[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'SystemMessage'], 'kwargs': {'content': '\nYou are an autonomous software engineer tasked with solving coding issues. Your role is to coordinate between code analysis and editing tasks. Follow these guidelines:\nYou have access to the following tools:\n- FILETOOL_OPEN_FILE: Use this to open and view file contents.\n- FILETOOL_GIT_REPO_TREE: Use this to view the repository structure.\n- FILETOOL_GIT_PATCH: Use this to generate patches for changes.\n\nDo the following steps in the same order:\n1. Issue Understanding:\n   - Carefully read and understand the given issue or bug report.\n   - Form a hypothesis about the problem and potential solutions.\n   - A workspace is initialized for you, and you will be working on workspace. \n   - The git repo is cloned in the path and you need to work in this directory.\n   - MAKE SURE THE EXISTING FUNCTIONALITY ISN\'T BROKEN BY SOLVING THE ISSUE, THAT IS, \n     THE SOLUTION

In [73]:
_df_sorted


,trace_id,name,messages,llm_output,run,generations,start_time,error,prompt_tokens,completion_tokens,total_tokens
9fc280d8-2321-45cc-a22f-c1f4b87e1d38,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 2878, 'completion_...",NaN,"[{'text': 'To start addressing this issue, I n...",2024-09-24 20:20:44.605148,None,2878,38,2916
1dbd497d-3ae8-4fe8-80c4-b67cbafaf19c,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 2217, 'completion_...",NaN,"[{'text': '', 'generation_info': None, 'type':...",2024-09-24 20:20:47.557885,None,2217,173,2390
6b444190-74f3-4779-9c8f-b36fb69f6e3f,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 2944, 'completion_...",NaN,"[{'text': '', 'generation_info': None, 'type':...",2024-09-24 20:20:53.398608,None,2944,170,3114
7a659d70-99ab-4699-9fe5-467b0cf7d90f,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 4356, 'completion_...",NaN,"[{'text': '', 'generation_info': None, 'type':...",2024-09-24 20:20:58.210305,None,4356,204,4560
9273dfde-a9e7-4d1a-a4fc-bfee0c857f57,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 5011, 'completion_...",NaN,[{'text': 'Based on the analysis of the `Prope...,2024-09-24 20:21:04.880151,None,5011,629,5640
149dbb98-061f-44dd-b8a9-e0697be251f4,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 6352, 'completion_...",NaN,[{'text': 'Thank you for the analysis. Now tha...,2024-09-24 20:21:14.802377,None,6352,37,6389
378b8b85-22d1-4536-9262-0ace4a309ec3,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 10025, 'completion...",NaN,"[{'text': '', 'generation_info': None, 'type':...",2024-09-24 20:21:17.760995,None,10025,128,10153
5dda694e-9708-4739-9e39-b41563da0e87,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 11533, 'completion...",NaN,"[{'text': '', 'generation_info': None, 'type':...",2024-09-24 20:21:21.696284,None,11533,145,11678
68a2d7f7-274c-471f-9ee5-5f03db3558c2,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 11730, 'completion...",NaN,"[{'text': '', 'generation_info': None, 'type':...",2024-09-24 20:21:27.694108,None,11730,99,11829
242a9043-f86d-4dd2-a441-eb0dcf8d9133,fd479cc6-ecf8-4551-8760-ca4638cc3582,BedrockChat,"[{'lc': 1, 'type': 'constructor', 'id': ['lang...","{'usage': {'prompt_tokens': 12105, 'completion...",NaN,"[{'text': '', 'generation_info': None, 'type':...",2024-09-24 20:21:31.523509,None,12105,520,12625


In [78]:
with open("messages.json", "w") as f:
    json.dump(_df_sorted.iloc[len(_df_sorted)-1]["messages"], f, indent=2)
with open("llm_output.json", "w") as f:
    json.dump(_df_sorted.iloc[len(_df_sorted)-1]["llm_output"], f, indent=2)




In [79]:
len(_df_sorted.iloc[len(_df_sorted)-1]["messages"])

74